In [ ]:
def inception_module_naive(layer,color_channels,kernel_output):
    mu=0
    sigma=0.1
    # 1*1 Convolution Layer
    filter_1_1=tf.Variable(tf.truncated_normal([1,1,color_channels,kernel_output], mu, sigma))
    block_conv1=tf.nn.conv2d(layer,filter_1_1,strides=[1,1,1,1], padding='SAME')
    
    # 3*3 Convolution Layer
    filter_3_1=tf.Variable(tf.truncated_normal([3,3,color_channels,kernel_output], mu, sigma))
    block_conv3=tf.nn.conv2d(layer,filter_3_1,strides=[1,1,1,1], padding='SAME')
    
    # 5*5 Convolution Layer
    filter_5_1=tf.Variable(tf.truncated_normal([5,5,color_channels,kernel_output], mu, sigma))
    block_conv5=tf.nn.conv2d(layer,filter_5_1,strides=[1,1,1,1], padding='SAME')
    
    # Average Pooling followed by 1*1 Convolution Layer
    average_pool=tf.nn.avg_pool(layer,ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')
    
    filter_1_2=tf.Variable(tf.truncated_normal([1,1,color_channels,kernel_output],mu,sigma))
    block_avgpool_conv1=tf.nn.conv2d(average_pool,filter_1_2,strides=[1,1,1,1], padding='SAME')
    
    # Concatenating layers
    bias=tf.Variable(tf.truncated_normal([color_channels+3*kernel_output], mu, sigma))
    x=tf.concat([block_conv1,block_conv3,block_conv5,block_avgpool],axis=3)
    x=tf.nn.bias_add(x,bias)
    
    return tf.nn.relu(x)
    

In [ ]:
def inception_module_googlenet(layer):
    mu=0
    sigma=0.1
    
    def convolution(layer,weight,bias):
        # W- Weight [Filter height, Filter width, color_channels, k_output]
        temp=tf.nn.conv2d(layer,weight,strides=[1,1,1,1],padding='SAME')
        return tf.nn.bias_add(temp,bias)
    
    def maxpool_3x3(layer):
        return tf.nn.max_pool(layer,ksize=[1,3,3,1],strides=[1,1,1,1],padding='SAME')
    
    def relu(layer):
        return tf.nn.relu(layer)
        
    # Weights,Biases
    # Three 1*1 Convolution Layers- Block One
    b1_conv_weight_1x1_1=tf.Variable(tf.truncated_normal([1,1,1,32],mu,sigma))
    b1_conv_bias_1x1_1=tf.Variable(tf.zeros([32]))
    
    b1_conv_weight_1x1_2=tf.Variable(tf.truncated_normal([1,1,1,16],mu,sigma))
    b1_conv_bias_1x1_2=tf.Variable(tf.zeros([16]))
    
    b1_conv_weight_1x1_3=tf.Variable(tf.truncated_normal([1,1,1,16],mu,sigma))
    b1_conv_bias_1x1_3=tf.Variable(tf.zeros([16]))
    
    # 3*3  Convolution Layer that follows b_1_conv_2- Block Two
    b2_conv_weight_3x3=tf.Variable(tf.truncated_normal([3,3,16,32],mu,sigma))
    c=tf.Variable(tf.zeros([32]))
    
    # 5*5 Convolution Layer that follows b1_conv_3- Block Two
    b2_conv_weight_5x5=tf.Variable(tf.truncated_normal([5,5,16,32],mu,sigma))
    b2_conv_bias_5x5=tf.Variable(tf.zeros([32]))
    
    # 1*1 Convolution Layer that follows a 3*3 maxpool layer- Block Two
    b2_conv_weight_1x1=tf.Variable(tf.truncated_normal([1,1,1,32],mu,sigma))
    b2_conv_bias_1x1=tf.Variable(tf.zeros([32]))
    
    
    # Fitting blocks
    # Parallel sub-modules in Block One
    b1_conv1_1x1=convolution(layer,b1_conv_weight_1x1_1,b1_conv_bias_1x1_1)
    b1_conv2_1x1=relu(convolution(layer,b1_conv_weight_1x1_2,b1_conv_bias_1x1_2))
    b1_conv3_1x1=relu(convolution(layer,b1_conv_weight_1x1_3,b1_conv_bias_1x1_2))
    b1_maxpool_3x3=maxpool_3x3(layer)
    
    # Parallel sub-modules in Block Two
    # 3*3 convolution connected to the second 1*1 convolution in Block One
    b2_conv1_3x3=convolution(b1_conv2_1x1,b2_conv_weight_3x3,b2_conv_weight_3x3)
    # 5*5 convolution connected to the third 1*1 convolution in Block One
    b2_conv1_5x5=convolution(b1_conv3_1x1,b2_conv_weight_5x5,b2_conv_bias_5x5)
    # 1*1 convolution connected to the max pool layer in Block One
    b2_conv1_1x1=convolution(b1_maxpool_3x3,b2_conv_weight_1x1,b2_conv_bias_1x1)
    
    # Concatenating modules b1_conv1_1x1 in Block One and all the modules in Block Two
    b3_concat=tf.concat([b1_conv1_1x1,b2_conv1_3x3,b2_conv1_3x3,b3_conv1_5x5,b2_conv1_1x1],3)
    
    inception=relu(b3_concat)
    
    return inception
    